In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
import nltk
import json
import re 
import string

from nltk.tag.stanford import StanfordNERTagger
ner_tagger = StanfordNERTagger("C:/Users/Eric/stanford-ner-2014-06-16/classifiers/english.all.3class.distsim.crf.ser.gz",
                               "C:/Users/Eric/stanford-ner-2014-06-16/stanford-ner-3.4.jar",
                                encoding="utf")



In [ ]:
text = 'While in France, Christine Lagarde discussed short-term stimulus efforts in a recent interview with the Wall Street Journal.'

tokenized_text = nltk.word_tokenize(text)
classified_text = ner_tagger.tag(tokenized_text)


In [ ]:
classified_text

In [ ]:
print(("\xc2"))

In [ ]:
text = "\nPlease enable Javascript to watch this video (Video may not load with Internet Explorer. Use Chrome or Firefox)\n \n\t\t\t\t\t\t\t\tGeorgia native Kobi Simmons is the highest ranked high school player on the US Select Team, and while he flashes tremendous potential, he has not been particularly dominant here in Treviso.  Making poor decisions with the ball trying to be too flashy, settling for floaters while avoiding contact, and not making the simple play at times this week, Simmons was better on day three, scoring 13 points while dishing out 4 assists.  His lack of strength has limited him a bit against the more mature, organized defenses he's faced.  His quickness is an obvious asset, especially when he asserts himself defending on the ball, as is his ability to play above the rim and knock down shots off the bounce, but it appears that he's still learning how to consistently exploit those tools against quality competition.  This isn't the easiest setting for a player like Simmons to excel in, as his best days are clearly ahead of him, so it was nice to see him find a rhythm in his last game in Italy.\n\t\t\t\t\t\t\t \nJonathan GivonyRecruiting Rankings: Scout #7, 24/7 #25, Rivals #15, ESPN #29Strengths:-Great size for his position-Smooth, fluid athlete-Quick in transition -Very good first step driving left -Puts pressure on the defense with his probing, aggressive style of ball-handling -Can make an outside shot-Shot 81% from the free throw line at the adidas Gauntlet (87 attempts)-Quick hands for steals -Anticipates in passing lanes -Very good potential defensively thanks to his sizeWeaknesses:-Still learning the nuances of how to run a team efficiently-Somewhat wild in his approach to the game-More of a scorer than a facilitator at this stage (-1.7 PPR in Gauntlet)-Shows average vision in the half-court-Lack of strength makes it difficult to finish around the rim effectively in half-court settings-Not a consistent 3-point shooter-Very low release point on his jumper-Has an average wingspan relative to his heightOutlook: Talented guard who's reputation revolves more around his potential as a big point than it does around his actual production. Still learning how to maximize his tools on both ends of the floor and play efficient basketball. Only 17 so still has plenty of time.\n\t\t\t\t\t\t\t \nComplexEmbed.Application.initialize({cId: \"xxZ29pbzqoqlrnVxQb4kZvfiRx-B7tTx\",adSetCode: \"d4063ffb5eea4665ac55cd9182bbb23e\",site: \"draftexpress\",kw: \"\"});Please enable Javascript to watch this video\n"
sents = nltk.sent_tokenize(text)
for sent in sents:
    tokens = nltk.word_tokenize(sent)
    classified = ner_tagger.tag(tokens)
    print(classified)

In [ ]:
printable = set(string.printable)
def clean_player_jsons(player_dict, output_filename):
    new_player_dict = {}
    for k, v in player_dict.items():
#         v = unicode(v, errors="ignore")
        sents = nltk.sent_tokenize(v)
        new_text = []
        for sent in sents:
            if "Javascript" not in sent and "Chrome" not in sent and "Firefox" not in sent:
#             if "Weaknesses-Only" in sent and "Javascript" not in sent and "Chrome" not in sent and "Firefox" not in sent:
#                 sent = sent.encode('utf-8').strip()
#                 print(sent)
    
                filtered_sent = filter(lambda x: x in printable, sent)
#                 print(filtered_sent)
        
                tokens = nltk.word_tokenize(filtered_sent)
#                 tokens = sent.split(r"[\s|-]")
#                 tokens = [token for token in tokens if "\\" not in token]
                print(tokens)
                classified_text = ner_tagger.tag(tokens)
                new_text.append(" ".join([word for (word, classification) in classified_text if classification != u"PERSON"]))
        new_player_dict[k] = " ".join(new_text)
    with open(output_filename, "w") as f:
        json.dump(new_player_dict, f)
    

In [2]:
with open("cleaned_prospects_to_docs.json", "r") as f:
    prospects = json.load(f)

In [3]:
with open("cleaned_curr_players_to_docs.json", "r") as f:
    curr_players = json.load(f)

In [ ]:
print("\xc2")
print("\xbc")
# print('\xc2\xbc'.encode("utf-8"))
text = "Shows the potential to guard multiple positionsWeaknesses-Only measured 6'6 ¼ in shoes by USA Basketball."
# print(u"Shows the potential to guard multiple positionsWeaknesses-Only measured 6'6 ¼ in shoes by USA Basketball.".split(r" "))
printable = set(string.printable)
filtered_text = filter(lambda x: x in printable, text)
print(nltk.word_tokenize(filtered_text))

In [ ]:
tokens = [u'Shows', u'the', u'potential', u'to', u'guard', u'multiple', u'positionsWeaknesses-Only', u'measured', u"6'6", u'\xbc', u'in', u'shoes', u'by', u'USA', u'Basketball', u'.']
regex = re.compile(r"[^[[:ascii:]]]")
filtered = [token for token in tokens if not regex.search(token)]
filtered

In [4]:
def get_tfidf_matrix(player_dict):
    text_list = []
    playername_list = []
    for player, text in player_dict.items():
        text_list.append(text)
        playername_list.append(player)

    tfidf_vectorizer = TfidfVectorizer(input=text_list, lowercase=True, stop_words="english", max_df=0.9, min_df=2)
    tfidf_text = tfidf_vectorizer.fit_transform(text_list)
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()
    return tfidf_text, tfidf_feature_names

In [5]:
tfidf_prospects_matrix, tfidf_prospects_feature_names = get_tfidf_matrix(prospects)
print(tfidf_prospects_matrix.shape)

tfidf_curr_players_matrix, tfidf_curr_players_feature_names = get_tfidf_matrix(curr_players)
print(tfidf_curr_players_matrix.shape)


(100, 2830)
(719, 9991)


In [6]:
def get_tf_matrix(player_dict):
    text_list = []
    playername_list = []
    for player, text in player_dict.items():
        text_list.append(text)
        playername_list.append(player)

    tf_vectorizer = CountVectorizer(input=text_list, lowercase=True, stop_words="english", max_df=0.9, min_df=2)
    tf_text = tf_vectorizer.fit_transform(text_list)
    tf_feature_names = tf_vectorizer.get_feature_names()
    return tf_text, tf_feature_names

In [7]:
tf_prospects_matrix, tf_prospects_feature_names = get_tf_matrix(prospects)
print(tf_prospects_matrix.shape)

tf_curr_players_matrix, tf_curr_players_feature_names = get_tf_matrix(curr_players)
print(tf_curr_players_matrix.shape)

(100, 2830)
(719, 9991)


In [8]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])
    

In [9]:
def nmf_topic_model(tfidf_matrix, tfidf_feature_names, num_topics=10):
    nmf_model = NMF(n_components=num_topics, random_state=1,alpha=.1, l1_ratio=.5).fit(tfidf_matrix)
    display_topics(nmf_model, tfidf_feature_names, no_top_words=15)

In [10]:
def lda_topic_model(tf_matrix, tf_feature_names, num_topics=10):
    lda_model = LatentDirichletAllocation(n_topics=num_topics).fit(tf_matrix)
    display_topics(lda_model, tf_feature_names, no_top_words=15)

In [11]:
nmf_topic_model(tfidf_prospects_matrix, tfidf_prospects_feature_names, num_topics=20)

Topic 0:
ball season game level ability does rim shot year size good high draft point forward
Topic 1:
draft prospects video schmitz 12 conference big express pac scouting previews east report section dx
Topic 2:
interview summit hoop nike practice drills day recap usa international measurements select tai wynyard team
Topic 3:
vs breakdown analysis video matchup strengths weaknesses european cal french kansas kentucky evolution u18 louisville
Topic 4:
zone eurocamp facet faces faced face eyes eye eyblweaknesses eybl extremely extreme extraordinary extraordinarily extra
Topic 5:
guard point lead ball self zone extreme face eyes eye eyblweaknesses eybl extremely extraordinarily extraordinary
Topic 6:
thomson draft arizona video colorado express archive matchup closer takes check look digital format twitter
Topic 7:
kansas 12 given season catch nations 63 valuable ball professional extended facet faces faced face
Topic 8:
redshirt ranked applebome producing revelation eyes sitting rhode 

In [12]:
lda_topic_model(tf_prospects_matrix, tf_prospects_feature_names, num_topics=20)

C:\Users\Eric\Anaconda2\envs\cs4300\lib\site-packages\sklearn\decomposition\online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


Topic 0:
post minutes 40 season game level big does rim shot perimeter ball ability point size
Topic 1:
draft season ball game size year able forward shooting play length high make just shot
Topic 2:
interview ball guard season draft year team video point size game floor kansas shooting long
Topic 3:
point carolina season south make ball line strength just attempts frame forward making game minutes
Topic 4:
interview guard team ball draft point nike wingspan summit hoop shooting frame game usa level
Topic 5:
draft ball year season video level team big game play wingspan point body floor does
Topic 6:
level draft high ball player ability times game shot forward frame shots point perimeter 40
Topic 7:
size draft high ball forward game does 10 level big perimeter time frame times make
Topic 8:
point season game guard year ability video shooting draft ball level long vs points team
Topic 9:
level shot season draft video game good big team ball shots long year scouts court
Topic 10:
offensi

In [13]:
nmf_topic_model(tfidf_curr_players_matrix, tfidf_curr_players_feature_names)

Topic 0:
season shot ability post level basket time does floor shots team just man offensive draft
Topic 1:
interview workout video angelo draft pre alan videos upshaw interviews connell christmasn chasson footage outtakes
Topic 2:
guard point guards shooting team court assists dribble offense vision teammates lane half making roll
Topic 3:
adjusted pace 40 minutes level season does jump 100 post defensive attempts junior offensive synergy
Topic 4:
video scouting report breakdown schmitz group syracuse reports givony lavine dermott season possessions artem klimenko
Topic 5:
vs analysis video videos breakdown matchup arizona schmitz kentucky compilation angelo section barcelona produced situational
Topic 6:
european euroleague acb spanish team europe adriatic league partizan serbian madrid international turkish age championships
Topic 7:
think going just like know givony guys ve really lot people reporter don got kind
Topic 8:
application embed initialize mc complex dermott 2014 reports

In [14]:
lda_topic_model(tfidf_curr_players_matrix, tfidf_curr_players_feature_names)

C:\Users\Eric\Anaconda2\envs\cs4300\lib\site-packages\sklearn\decomposition\online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


Topic 0:
season shot point ability level video team post draft does guard just time shooting basket
Topic 1:
video scouting report breakdown season ability point team shot vs just level guard does draft
Topic 2:
season shot point kn team guard level state floor video did basketball ability draft best
Topic 3:
today team did season villanova didn point shot way lowry ucla shooting basket right wasn
Topic 4:
point just shot team guard time season like video interview half workout got really givony
Topic 5:
video interview vs analysis workout shooting point ability team shot draft level just does end
Topic 6:
season bryant post shooting ability point guard shot does level shots play big interview defense
Topic 7:
point guard ability season just shot greek size scoring play team does level end villanova
Topic 8:
memphis wall nice colorado raised ended leads post plausible telekom instead matures expending vitals reduced
Topic 9:
workout interview dribble bowling complacent brannen tempo po